In [7]:
!pip install --no-deps --force-reinstall 'https://github.com/bitsandbytes-foundation/bitsandbytes/releases/download/continuous-release_main/bitsandbytes-1.33.7.preview-py3-none-manylinux_2_24_x86_64.whl'
!pip uninstall apex
# vllm doesn't validate rocm. need to check further.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 117.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.48.0.dev0
    Uninstalling bitsandbytes-0.48.0.dev0:
      Successfully uninstalled bitsandbytes-0.48.0.dev0

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [1]:
import gc
from typing import Optional

import torch
from huggingface_hub import snapshot_download

from vllm import EngineArgs, LLMEngine, RequestOutput, SamplingParams
from vllm.lora.request import LoRARequest

INFO 09-27 10:17:16 [__init__.py:244] Automatically detected platform rocm.


In [2]:
def create_test_prompts(
    lora_path: str,
) -> list[tuple[str, SamplingParams, Optional[LoRARequest]]]:
    return [
        # this is an example of using quantization without LoRA
        (
            "My name is",
            SamplingParams(
                temperature=0.0, logprobs=1, prompt_logprobs=1, max_tokens=128
            ),
            None,
        ),
        # the next three examples use quantization with LoRA
        (
            "my name is",
            SamplingParams(
                temperature=0.0, logprobs=1, prompt_logprobs=1, max_tokens=128
            ),
            LoRARequest("lora-test-1", 1, lora_path),
        ),
        (
            "The capital of USA is",
            SamplingParams(
                temperature=0.0, logprobs=1, prompt_logprobs=1, max_tokens=128
            ),
            LoRARequest("lora-test-2", 1, lora_path),
        ),
        (
            "The capital of France is",
            SamplingParams(
                temperature=0.0, logprobs=1, prompt_logprobs=1, max_tokens=128
            ),
            LoRARequest("lora-test-3", 1, lora_path),
        ),
    ]

In [3]:
def process_requests(
    engine: LLMEngine,
    test_prompts: list[tuple[str, SamplingParams, Optional[LoRARequest]]],
):
    """Continuously process a list of prompts and handle the outputs."""
    request_id = 0

    while test_prompts or engine.has_unfinished_requests():
        if test_prompts:
            prompt, sampling_params, lora_request = test_prompts.pop(0)
            engine.add_request(
                str(request_id), prompt, sampling_params, lora_request=lora_request
            )
            request_id += 1

        request_outputs: list[RequestOutput] = engine.step()
        for request_output in request_outputs:
            if request_output.finished:
                print("----------------------------------------------------")
                print(f"Prompt: {request_output.prompt}")
                print(f"Output: {request_output.outputs[0].text}")

In [4]:
def initialize_engine(
    model: str, quantization: str, lora_repo: Optional[str]
) -> LLMEngine:
    """Initialize the LLMEngine."""

    engine_args = EngineArgs(
        model=model,
        quantization=quantization,
        enable_lora=True,
        max_lora_rank=64,
        max_loras=4,
    )
    return LLMEngine.from_engine_args(engine_args)

In [5]:
def main():
    """Main function that sets up and runs the prompt processing."""

    test_configs = [
        # QLoRA (https://arxiv.org/abs/2305.14314)
        # {
        #     "name": "qlora_inference_example",
        #     "model": "huggyllama/llama-7b",
        #     "quantization": "bitsandbytes",
        #     "lora_repo": "timdettmers/qlora-flan-7b",
        # },
        {
            "name": "AWQ_inference_with_lora_example",
            "model": "TheBloke/TinyLlama-1.1B-Chat-v0.3-AWQ",
            "quantization": "awq",
            "lora_repo": "jashing/tinyllama-colorist-lora",
        },
        {
            "name": "GPTQ_inference_with_lora_example",
            "model": "TheBloke/TinyLlama-1.1B-Chat-v0.3-GPTQ",
            "quantization": "gptq",
            "lora_repo": "jashing/tinyllama-colorist-lora",
        },
    ]

    for test_config in test_configs:
        print(f"~~~~~~~~~~~~~~~~ Running: {test_config['name']} ~~~~~~~~~~~~~~~~")
        engine = initialize_engine(
            test_config["model"], test_config["quantization"], test_config["lora_repo"]
        )
        lora_path = snapshot_download(repo_id=test_config["lora_repo"])
        test_prompts = create_test_prompts(lora_path)
        process_requests(engine, test_prompts)

        # Clean up the GPU memory for the next test
        # del engine
        # gc.collect()
        # torch.cuda.empty_cache()

In [6]:
main()

~~~~~~~~~~~~~~~~ Running: AWQ_inference_with_lora_example ~~~~~~~~~~~~~~~~
INFO 09-27 10:17:54 [config.py:853] This model supports multiple tasks: {'classify', 'generate', 'embed', 'score', 'reward'}. Defaulting to 'generate'.
INFO 09-27 10:17:54 [config.py:1467] Using max model len 2048
WARNING 09-27 10:18:04 [rocm.py:346] Using AWQ quantization with ROCm, but VLLM_USE_TRITON_AWQ is not set, enabling VLLM_USE_TRITON_AWQ.
WARNING 09-27 10:18:04 [config.py:962] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 09-27 10:18:04 [config.py:2267] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 09-27 10:18:04 [config.py:4566] full_cuda_graph is not supported with cascade attention. Disabling cascade attention.
WARNING 09-27 10:18:04 [utils.py:2613] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-mult

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  3.34it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  3.34it/s]



INFO 09-27 10:18:20 [default_loader.py:272] Loading weights took 0.33 seconds
INFO 09-27 10:18:20 [punica_selector.py:19] Using PunicaWrapperGPU.
INFO 09-27 10:18:20 [gpu_model_runner.py:1782] Model loading took 1.1875 GiB and 0.758358 seconds


Process EngineCore_0:
Traceback (most recent call last):
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.12/dist-packages/vllm/v1/engine/core.py", line 523, in run_engine_core
    raise e
  File "/usr/local/lib/python3.12/dist-packages/vllm/v1/engine/core.py", line 510, in run_engine_core
    engine_core = EngineCoreProc(*args, **kwargs)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/vllm/v1/engine/core.py", line 394, in __init__
    super().__init__(vllm_config, executor_class, log_stats,
  File "/usr/local/lib/python3.12/dist-packages/vllm/v1/engine/core.py", line 82, in __init__
    self._initialize_kv_caches(vllm_config)
  File "/usr/local/lib/python3.12/dist-packages/vllm/v1/engine/core.py", line 142, in _initialize_kv_caches
    avail

ERROR 09-27 10:18:21 [core.py:519] EngineCore failed to start.
ERROR 09-27 10:18:21 [core.py:519] Traceback (most recent call last):
ERROR 09-27 10:18:21 [core.py:519]   File "/usr/local/lib/python3.12/dist-packages/vllm/v1/engine/core.py", line 510, in run_engine_core
ERROR 09-27 10:18:21 [core.py:519]     engine_core = EngineCoreProc(*args, **kwargs)
ERROR 09-27 10:18:21 [core.py:519]                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ERROR 09-27 10:18:21 [core.py:519]   File "/usr/local/lib/python3.12/dist-packages/vllm/v1/engine/core.py", line 394, in __init__
ERROR 09-27 10:18:21 [core.py:519]     super().__init__(vllm_config, executor_class, log_stats,
ERROR 09-27 10:18:21 [core.py:519]   File "/usr/local/lib/python3.12/dist-packages/vllm/v1/engine/core.py", line 82, in __init__
ERROR 09-27 10:18:21 [core.py:519]     self._initialize_kv_caches(vllm_config)
ERROR 09-27 10:18:21 [core.py:519]   File "/usr/local/lib/python3.12/dist-packages/vllm/v1/engine/core.py", line 142, in _init

[rank0]:[W927 10:18:21.369484922 ProcessGroupNCCL.cpp:1476] Warning: WARNING: destroy_process_group() was not called before program exit, which can leak resources. For more info, please see https://pytorch.org/docs/stable/distributed.html#shutdown (function operator())


RuntimeError: Engine core initialization failed. See root cause above. Failed core proc(s): {}